# 3.4 Nonlinear minimization problems
We consider minimization problems of the form

$$\text{find } u \in V \text{ s.t. } E(u) \leq E(v) \quad \forall~  v \in V.$$

In [ ]:
# imports
from ngsolve import *
from ngsolve.webgui import Draw

Similar to the previous unit we want solve the (minimization) problem using Newton's method. However this time we don't start with an equation but a minimization problem. We will let `NGSolve` derive the corresponding expressions for minimization conditions.

To solve the problem we use the `Variation` integrator of `NGSolve` and formulate the problem through the symbolic description of an energy functional. 

Let $E(u)$ be the energy that is to be minimized for the unknown state $u$. 

Then a necessary optimality condition is that the derivative at the minimizer $u$ in all directions $v$ vanishes, i.e. 
$$
  \delta E(u) (v) = 0 \quad \forall v \in V
$$

At this point we are back in business of the previous unit as we now have an equation that we need to solve.

Let's continue with a concrete example.

## Scalar minimization problems
As a first example we take $V = H^1_0$ and 

$$
E(u) = \int_{\Omega} \frac12 \vert \nabla u \vert^2 + \frac{1}{12} u^4 - fu ~dx.
$$

The minimization is equivalent (due to convexity) to solving the nonlinear PDE (cf. [unit 3.3](../unit-3.3-nonlinear/nonlinear.ipynb) with $f=10$)

$$
 \delta E(u)(v) = 0 ~ \forall v \in V \quad \Leftrightarrow \quad - \Delta u + \frac13 u^3 = f \text{ in } V'
$$

with $\Omega = (0,1)^2$.

In [ ]:
# define geometry and generate mesh
from ngsolve import *
from ngsolve.webgui import *
from netgen.occ import *
shape = Rectangle(1,1).Face()
shape.edges.Min(X).name="left"
shape.edges.Max(X).name="right"
shape.edges.Min(Y).name="bottom"
shape.edges.Max(Y).name="top"
geom = OCCGeometry(shape, dim=2)
mesh = Mesh(geom.GenerateMesh(maxh=0.3))

We solve the PDE with a Newton iteration.

In [ ]:
V = H1(mesh, order=4, dirichlet=".*")
u = V.TrialFunction()

We define the semi-linear form expression through the energy functional using the `Variation`-keyword:

In [ ]:
a = BilinearForm (V, symmetric=True)
a += Variation ( (0.5*grad(u)*grad(u) + 1/12*u**4-10*u) * dx)

Now `NGSolve` applies the derivative on the functional so that the previous statement corresponds to:
```
a += (grad(u) * grad(v) + 1/3*u**3*v - 10 * v)*dx
``` 
(which has the same form as the problems in [the nonlinear example](../unit-3.7-nonlinear/nonlinear.ipynb))

We recall the Newton iteration (cf. [unit-3.3](../unit-3.3-nonlinear/nonlinear.ipynb) ) and now apply the same loop:

- Given an initial guess $u^0$
- loop over $i=0,..$ until convergence:
  - Compute linearization: $A (u^i) + \delta A(u^i) \Delta u^{i} = 0 $:
    - $f^i = A (u^i)$ 
    - $B^i = \delta A(u^i)$ 
    - Solve $B^i \Delta u^i = -f^i$
  - Update $u^{i+1} = u^i + \Delta u^{i}$
  - Evaluate stopping criteria
- Evaluate $E(u^{i+1})$

As a stopping criteria we take $\langle A u^i,\Delta u^i \rangle = \langle A u^i, A u^i \rangle_{(B^i)^{-1}}< \varepsilon$.

Note that $A(u^i)$ (`a.Apply(...)`) and $\delta A(u^i)$ (`a.AssembleLinearization(...)`) are now derived from $A$ which is defined as $A = \delta E$ through the energy functional $E(\cdot)$. 

We obtain a similar Newton solver with the two additional advantages:

 * We don't have to form $\delta E$ manually, but let `NGSolve` do the job and
 * we can use the energy functional to interprete the success of iteration steps  

In [ ]:
def SolveNonlinearMinProblem(a,gfu,tol=1e-13,maxits=10, callback=lambda gfu: None):
    res = gfu.vec.CreateVector()
    du  = gfu.vec.CreateVector()
    callback(gfu)
    for it in range(maxits):
        print ("Newton iteration {:3}".format(it),
               ", energy = {:16}".format(a.Energy(gfu.vec)),end="")
    
        #solve linearized problem:
        a.Apply (gfu.vec, res)
        a.AssembleLinearization (gfu.vec)
        du.data = a.mat.Inverse(V.FreeDofs()) * res
    
        #update iteration
        gfu.vec.data -= du
        callback(gfu)

        #stopping criteria
        stopcritval = sqrt(abs(InnerProduct(du,res)))
        print ("<A u",it,", A u",it,">_{-1}^0.5 = ", stopcritval)
        if stopcritval < tol:
            break
        Redraw(blocking=True)

So, let's try it out:

In [ ]:
gfu = GridFunction(V)
gfu.Set((x*(1-x))**4*(y*(1-y))**4) # initial guess
gfu_it = GridFunction(gfu.space,multidim=0)
cb = lambda gfu : gfu_it.AddMultiDimComponent(gfu.vec) # store current state
SolveNonlinearMinProblem(a,gfu, callback = cb)
print ("energy = ", a.Energy(gfu.vec))    

In [ ]:
Draw(gfu,mesh,"u", deformation = True)
#Draw(gfu_it,mesh,"u", deformation = True)

Again, a Newton for minimization is also shipped with NGSolve (actually it is the same with the additional knowledge about the Energy and the possibility to do a line search for a given search direction):

In [ ]:
from ngsolve.solvers import *
gfu.Set((x*(1-x))**4*(y*(1-y))**4) # initial guess
NewtonMinimization(a,gfu)
#Draw(gfu,mesh,"u", deformation = True)

## Nonlinear elasticity

We consider a beam which is fixed on one side and is subject to gravity only. We assume a Neo-Hookean hyperelastic material. The model is a nonlinear minimization problem with 

$$
  E(v) := \int_{\Omega} \frac{\mu}{2} ( \operatorname{tr}(F^T F-I)+\frac{2 \mu}{\lambda} \operatorname{det}(F^T F)^{\frac{\lambda}{2\mu}} - 1) - \gamma ~ (f,v) ~~ dx
$$

where $\mu$ and $\lambda$ are the Lamé parameters and $F = I + D v$ where $v: \Omega \to \mathbb{R}^2$ is the sought for displacement.

We fix the domain to $\Omega = (0,1) \times (0,0.1)$

In [ ]:
# define geometry and generate mesh
shape = Rectangle(1,0.1).Face()
shape.edges.Min(X).name="left"
shape.edges.Min(X).maxh=0.01
shape.edges.Max(X).name="right"
shape.edges.Min(Y).name="bot"
shape.edges.Max(Y).name="top"
geom = OCCGeometry(shape, dim=2)
mesh = Mesh(geom.GenerateMesh(maxh=0.05))

In [ ]:
# E module and poisson number:
E, nu = 210, 0.2
# Lamé constants:
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

V = VectorH1(mesh, order=2, dirichlet="left")
u  = V.TrialFunction()

#gravity:
force = CoefficientFunction( (0,-1) )

Now, we recall the energy
$$
  E(v) := \int_{\Omega} \frac{\mu}{2} ( \operatorname{tr}(F^T F-I)+\frac{2 \mu}{\lambda} \operatorname{det}(F^T F)^{\frac{\lambda}{2\mu}} - 1) - \gamma ~ (f,v) ~~ dx
$$

In [ ]:
def Pow(a, b):
    return exp (log(a)*b)
    
def NeoHook (C):
    return 0.5 * mu * (Trace(C-I) + 2*mu/lam * Pow(Det(C), -lam/2/mu) - 1)

I = Id(mesh.dim)
F = I + Grad(u)
C = F.trans * F

factor = Parameter(1.0)

a = BilinearForm(V, symmetric=True)
a += Variation(  NeoHook (C).Compile() * dx 
                -factor * (InnerProduct(force,u) ).Compile() * dx)

We want to solve the minimization problem for $\gamma = 5$. Due to the high nonlinearity in the problem, the Newton iteration will not convergence for any initial guess. We approach the case $\gamma = 5$ by solving problems with $\gamma = i/10$ for $i=1,..,50$ and taking the solution of the previous problem as an initial guess.

In [ ]:
gfu = GridFunction(V)
gfu.vec[:] = 0
gfu_l = GridFunction(V,multidim=0)
gfu_l.AddMultiDimComponent(gfu.vec)
for loadstep in range(50):
    print ("loadstep", loadstep)
    factor.Set ((loadstep+1)/10)
    SolveNonlinearMinProblem(a,gfu)
    if (loadstep + 1) % 10 == 0:
        gfu_l.AddMultiDimComponent(gfu.vec)

In [ ]:
Draw(gfu_l,mesh, interpolate_multidim=True, animate=True, 
     deformation=True, min=0, max=1, autoscale=False)

## Supplementary 1: Allen-Cahn equation

The Allen-Cahn equations describe the process of phase separation and is the ($L^2$) gradient-flow equation to the energy
$$
  E(v) = \int_{\Omega} \varepsilon \vert \nabla v \vert^2~+~ \underbrace{v^2(1-v^2)}_{\Psi(v)} ~ dx
$$
where $\Psi(v)$ is the so-called double-well potential with the minima $-1,0,1$ (with $0$ being an unstable minima).



The solution to the Allen-Cahn equation solves

$$
\partial_t u = \frac{\delta E}{\delta u}
$$

The quantity $u$ is an indicator for a phase where $-1$ refers to one phase and $1$ to another phase. 

The equation has two driving forces:

- $u$ is pulled into one of the two stable minima states ($-1$ and $1$) of the nonlinear term $u^2(1-u^2)$ (separation of the phases)
- the diffusion term scaled with $\varepsilon$ enforces a smooth transition between the two phases. $\varepsilon$ determines the size of the transition layer



We use the `Energy` formulation for  energy minimization combined with a simple time stepping with an implicit Euler discretization:

$$
 M u^{n+1} - M u^n = \Delta t \underbrace{\frac{\delta E}{\delta u}}_{=:A(u)} (u^{n+1})
$$

which we can interprete as a nonlinear minimization problem again with the energy

$$
  E^{IE}(v) = \int_{\Omega} \frac{\varepsilon}{2} \vert \nabla v \vert^2~+~v^2(1-v^2) + \frac{1}{2\Delta t} \vert v - u^n \vert^2 ~ dx
$$


To solve the nonlinear equation at every time step we again rely on Newton's method.
We first define the periodic geometry, setup the formulation and then apply Newton's method in the next cells:

In [ ]:
# define periodic geometry and generate mesh
shape = Rectangle(1,1).Face()
right=shape.edges.Max(X)
right.name="right"
shape.edges.Min(X).Identify(right,name="left")
top=shape.edges.Max(Y)
top.name="top"
shape.edges.Min(Y).Identify(top,name="bottom")
geom = OCCGeometry(shape, dim=2)
mesh = Mesh(geom.GenerateMesh(maxh=0.1))


In [ ]:
#use a periodic fe space correspondingly
V = Periodic(H1(mesh, order=3))
u = V.TrialFunction()

eps = 4e-3
dt = Parameter(1e-1)
gfu = GridFunction(V)
gfuold = GridFunction(V)
a = BilinearForm (V, symmetric=False)
a += Variation( (eps/2*grad(u)*grad(u) + ((1-u*u)*(1-u*u)) 
                     + 0.5/dt*(u-gfuold)*(u-gfuold)) * dx)

In [ ]:
from math import pi
gfu = GridFunction(V)
#gfu.Set(sin(2*pi*x)) # regular initial values
gfu.Set(sin(1e7*(x+y*y))) #<- essentially a random function
gfu_t = GridFunction(V, multidim=0)
gfu_t.AddMultiDimComponent(0.1*gfu.vec)

In [ ]:
t = 0; tend = 5; cnt = 0; sample_rate = int(floor(0.5/dt.Get()))
while t < tend - 0.5 * dt.Get():
    gfuold.vec.data = gfu.vec
    SolveNonlinearMinProblem(a,gfu)
    if (cnt+1) % sample_rate == 0:
        gfu_t.AddMultiDimComponent(0.1*gfu.vec)
    t += dt.Get(); cnt += 1
    print("t = ", t)

In [ ]:
Draw(gfu_t, mesh, interpolate_multidim=True, animate=True,
     min=-0.1, max=0.1, autoscale=False, deformation=True)

## Supplementary 2: Minimal energy extension (postscript in [unit-2.1.3](../unit-2.1.3-bddc/bddc.ipynb) )

In [unit-2.1.3](../unit-2.1.3-bddc/bddc.ipynb) we discussed the BDDC preconditioner and characterized the coarse grid solution as the condensed problem with the continuity only w.r.t. the coarse grid dofs.

We can characterize this as a minimization problem involving

$$
  u \in V^{ho,disc}, \quad u^{lo,cont} \in V^{lo,cont}, \quad \lambda \in V^{lo,disc},
$$

Here $u$ is the solution to the coarse-grid (decoupled) problem where $u^{lo,cont}$ represents the coarse (continuity-)dofs and $\lambda$ is the corresponding Lagrange multiplier to enforce continuity of $u$ w.r.t. the `lo,cont`-dofs.


In [ ]:
mesh = Mesh(geom.GenerateMesh(maxh=0.1))
fes_ho = Discontinuous(H1(mesh, order=10))
fes_lo = H1(mesh, order=1, dirichlet=".*")
fes_lam = Discontinuous(H1(mesh, order=1))
fes = fes_ho*fes_lo*fes_lam
uho, ulo, lam = fes.TrialFunction()

The energy that is to be minimized is:

$$
\int_{\Omega} \frac12 \Vert \nabla u \Vert^2  - u + \sum_T \sum_{V \in V(T)} ((u-u^{lo})\cdot \lambda)|_{V} \longrightarrow \operatorname{min}!
$$

In [ ]:
a = BilinearForm(fes)
a += Variation(0.5 * grad(uho)*grad(uho)*dx 
               - 1*uho*dx 
               + (uho-ulo)*lam*dx(element_vb=BBND))
gfu = GridFunction(fes)
NewtonMinimization(a=a, u=gfu)
Draw(gfu.components[0],mesh,deformation=True)

The minimization problem is solved by the solution of the PDE:
$$
\int_{\Omega} \nabla u \cdot \nabla v = \int_{\Omega} 1 \cdot v \quad \forall ~ v \in V^{ho,disc}
$$
under the constraint
$$
  u(v) = u^{lo}(v) \quad \text{ for all vertices } v \in V(T) \text{ for all } T.
$$